In [110]:
import pygame
import math
from Body_Classes import Body

pygame.init()

display_width = 1000
display_height = 800

gameDisplay = pygame.display.set_mode((display_width,display_height))
pygame.display.set_caption('Sticky')

black = (0,0,0)
white = (255,255,255)

clock = pygame.time.Clock()

summon_width = 7


class Line:
    
    thickness = 2
    
    def __init__(self,nodes,color):
        self.nodes = list(nodes)
        self.color = tuple(color)
        
    def draw(self):
        pygame.draw.lines(gameDisplay, self.color, True, self.nodes, self.thickness)

class Shape:
    
    def __init__(self,nodes,color):
        self.nodes = list(nodes)
        self.color = tuple(color)
        self.nodes.append(self.nodes[0]) #wrap shape around so it will fill
        self.angle = 0 # angle for rotating image
        
    def draw(self,position,scale,direction):
        
        nodes = [tuple(i*scale*d+j for i,j,d in zip(k,position,direction)) for k in self.nodes]
       # print(nodes)
        pygame.draw.polygon(gameDisplay, self.color, nodes)

   # def draw(self,position,scale):
   #     self.draw(position,scale,[1,1])
        
    def rotate(self,angle): # rotate shape by set amount
        self.nodes = [(math.cos(angle)*n[0]-math.sin(angle)*n[1], math.cos(angle)*n[1]+math.sin(angle)*n[0]) for n in self.nodes]
        
    
    def set_angle(self,angle): # set angle using input
        self.rotate(angle-self.angle)
        
        
        
class Circle:
    def __init__(self,position,radius,color):
        self.pos = position
        self.color = tuple(color)
        self.radius = radius
        
    def draw(self,position,scale,width,direction):
        pygame.draw.circle(gameDisplay, self.color, [int(i*k+j) for i,j,k in zip(self.pos,position,direction)],int(self.radius*scale),int(width))

    def rotate(self,angle):
        self.pos = (math.cos(angle)*self.pos[0]-math.sin(angle)*self.pos[1], math.cos(angle)*self.pos[1]+math.sin(angle)*self.pos[0])
    
    def set_angle(self,angle): # set angle using input
        self.rotate(angle-self.angle)
        
        
class Arrow:
    def __init__(self,position,scale,color):
        self.pos = position
        self.s = scale
        self.base_shape = [(5,-10),(5,3),(10,3),(0,10),(-10,3),(-5,3),(-5,-10)]
        self.direction = [-1,0]
        self.shapes = Shape([(5,-10),(5,3),(10,3),(0,10),(-10,3),(-5,3),(-5,-10)],color)
        
    def switch_direction(self,direction):
        
        if direction != self.direction: # if change is needed
            
            if self.direction[0]:
                pass
        
class Stickman(Body):
    
    airborne = False
    sworded = False
    summoning = False
    running = False
    lashed = [0,0,0,0]
    angle = 0
    
    def __init__(self,scale,position,color):
        super().__init__(position,[20*scale,85*scale]) # Create & initialize location (x,y) and size (w,h)
        self.corporeal = True              
        self.s = scale
        self.pos = list(position)
        
        
        self.bodyparts = {'head_standing': Shape([(10,-10),(20,-30),(-10,-50),(0,-30),(-20,-30)],color)}
        self.bodyparts['arm1_running'] = Shape([(-10-1,5),(10-1,5),(2-21,50-9),(6-4,85-28),(-6,85-20),(-2-25,50-8)],[i*.9 for i in color])
        self.bodyparts['head_running'] = Shape([(10,-10),(20,-30),(-10,-50),(0,-30),(-20,-30)],color)
        self.bodyparts['body_running'] = Shape([(-20,0),(20,0),(0-10,70),(-20-20,140),(20,140),(15,105),(0-10,70)],color)
        self.bodyparts['body_standing'] = Shape([(-20,0),(20,0),(0,70),(-20,140),(20,140),(15,105),(0,70)],color)
        self.bodyparts['sword_standing'] = Shape([(10,88),(50,92),(130,85),(100,65),(10,72)],[(100,100,255)])
        d = [-34,46]
        self.bodyparts['sword_running'] = Shape([(88+d[0],-10+d[1]),(92+d[0],-50+d[1]),(85+d[0],-130+d[1]),(65+d[0],-100+d[1]),(72+d[0],-10+d[1])],[(100,100,255)])
        self.bodyparts['sword_hilt_standing'] = Shape([(-10,77),(-10,83),(10,83),(10,77)],[i*.6 for i in color])
        self.bodyparts['sword_hilt_running'] = Shape([(77+d[0],10+d[1]),(83+d[0],10+d[1]),(83+d[0],-10+d[1]),(77+d[0],-10+d[1])],[i*.6 for i in color])
        self.bodyparts['arm_standing'] = Shape([(-10,5),(10,5),(2,50),(6,85),(-6,85),(-2,50)],[i*.9 for i in color])
        self.bodyparts['arm2_running'] = Shape([(-10-3,5),(10-3,5),(2+20,50-8),(6+40,85-44),(-6+52,85-34),(-2+20,50-5)],[i*.9 for i in color])
        
        self.animations = {'summoning': Circle((0,80),summon_width,(100,100,255))}
        self.animations['summoning_running']= Circle((80+d[0],0+d[1]),summon_width,(100,100,255))
        self.direction = [1,1]
        
        shift = (0,-45)
        for b in self.bodyparts.values():
            b.nodes = [tuple(k[i]+shift[i] for i in range(2)) for k in b.nodes ]
        for b in self.animations.values():
            b.pos = tuple(b.pos[i]+shift[i] for i in range(2))
           
          
            
    def rotate(self,angle):
        self.angle += angle
        for b in self.bodyparts.values():
            b.rotate(angle)
        for b in self.animations.values():
            b.rotate(angle)
     
    def set_angle(self,angle): # set angle using input
        self.rotate(angle-self.angle)    
        
    def draw(self):
        self.running = (abs(self.vel[0])>0)
        if self.running:
            self.direction[0] = int(math.copysign(1,self.vel[0]))
          
        
        for k,v in self.bodyparts.items():
            if self.running == ('running' in k) and (self.sworded or not 'sword' in k) and (self.summoning or not 'summoning' in k):
                v.draw(self.pos,self.s,self.direction)
         
        if self.summoning:
            if self.summoning < 0:
                self.summoning = 0
            else:
                if self.running:
                    self.animations['summoning_running'].draw(self.pos,self.s*self.summoning,self.s*summon_width,self.direction)
                else:
                    self.animations['summoning'].draw(self.pos,self.s*self.summoning,self.s*summon_width,self.direction)

        
    def move(self): # move, and deal with internal state
        
        super().move()
        
        if self.summoning: # evolve summoning time
            self.summoning -= 0.2
        
            if self.summoning<=1:
                self.summoning = False
                self.sworded = True
                
    def jump(self):
        
        if not self.airborne:
            self.vel[1] = -20*self.direction[1]
            self.airborne = True

        
                
    
    
floor = 450
ceil = 100
wall1 = 50
wall2 = display_width-50

zeth = Stickman(1,(500,floor),white)  

#x = Line([(300,300),(200,200)],(255,0,0))




crashed = False


font = pygame.font.Font('freesansbold.ttf', 370) 
  
# create a text suface object, 
# on which text is drawn on it. 
text = font.render('ZETH', True, white) 
textRect = text.get_rect()  
# set the center of the rectangular object. 
textRect.center = (5+ display_width // 2, 370 +  display_height // 2) 





left_key = False
right_key = False
up_key = False
down_key = False
invert_key = False


while True:
    
    for event in pygame.event.get():

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                left_key = True
            elif event.key == pygame.K_RIGHT:
                right_key = True
             
                
            elif event.key == pygame.K_UP:
                up_key = True
            elif event.key == pygame.K_DOWN:
                down_key = True
                
            elif event.key == pygame.K_s:
                if not zeth.sworded:
                    zeth.summoning = 10
                
            elif event.key == pygame.K_x:
                zeth.sworded = False
                
            elif event.key == pygame.K_d:
                zeth.jump()
                
            elif event.key == pygame.K_a:
                invert_key = True
                zeth.set_angle(math.pi)
                #zeth.direction[1] = 1
                
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_key = False
            elif event.key == pygame.K_RIGHT:
                right_key = False
            elif event.key == pygame.K_s:
                zeth.summoning = False
            
            elif event.key == pygame.K_UP:
                up_key = False
            elif event.key == pygame.K_DOWN:
                down_key = False
            elif event.key == pygame.K_a:
                invert_key = False
                zeth.set_angle(0)
               # zeth.direction[1] = 1
        ######################  
   

    gameDisplay.fill(black)

    zeth.vel[1] += 0.7*(1-2*invert_key)
    zeth.move()
    zeth.airborne = True
    if zeth.pos[1] >= floor:
        zeth.pos[1] = floor
        zeth.airborne = False
        zeth.vel[1] = 0
    
    
    if zeth.pos[1] <= ceil:
        zeth.pos[1] = ceil
        zeth.airborne = False
        zeth.vel[1] = 0
    
    zeth.vel[0] = 5*(right_key-left_key)

   # x.draw()
    zeth.draw()
    
    gameDisplay.blit(text, textRect) 
        
    
    pygame.display.flip()
    clock.tick(100)
    
    
    
 
    


KeyboardInterrupt: 

In [6]:
cd

/Users/Sam


In [2]:
[0,1] == [0,1]

True

In [4]:
(2+20,50-8),(6+40,85-44),(-6+52,85-34),(-2+20,50-5)

((22, 42), (46, 41), (46, 51), (18, 45))

In [7]:
math.cos()

-0.9999987317275395

In [9]:
math.cos(math.pi)

-1.0

In [1]:
import math, numpy as np, pygame


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
np.add(tuple((1,1)),tuple((3,4)))

array([4, 5])

In [4]:
math

<module 'math' from '/Users/Sam/anaconda3/lib/python3.7/lib-dynload/math.cpython-37m-darwin.so'>

In [12]:
x = np.array((1,1))
y = np.array((3,3))

In [9]:
x*y

array([3, 3])

In [18]:
isinstance(x,np.ndarray)

True

In [16]:
type(np.array([0,0]))

numpy.ndarray

In [22]:
-0 != -0.0

False

In [27]:
x = np.array([[1,1],[1,1]])

In [30]:
x[:]

array([[1, 1],
       [1, 1]])

In [40]:
list(float)

TypeError: 'type' object is not iterable

In [37]:
type(1.0)

float

In [42]:
abs(-x)

array([[1, 1],
       [1, 1]])

In [56]:
times = 4

In [60]:
rot90mat = np.array([[0,-1],[1,0]])


In [62]:
np.matmul(x,rot90mat)

array([[ 1, -1],
       [ 1, -1]])

In [106]:
x = np.array([[1,2],[3,4]])

In [107]:
y = x

In [108]:
x[1,:] *= 2

In [109]:
x,y

(array([[1, 2],
        [6, 8]]), array([[1, 2],
        [6, 8]]))

In [100]:
y

array([[1, 2],
       [3, 4]])

In [80]:
x = None
x is None

True

In [84]:
tuple(x[1])

(3, 4)

In [86]:
isinstance(tuple([1]),tuple)

True

In [87]:
np.matrix_det(x)

AttributeError: module 'numpy' has no attribute 'matrix_det'

In [89]:
x = list()

In [90]:
x

[]

TypeError: 'type' object is not iterable